# LangChain: Introduction and Getting Started

### LangChain
At its core, LangChain is a framework built around LLMs. We can use it for chatbots, Generative Question-Answering (GQA), summarization, and much more.

The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs. Chains may consist of multiple components from several modules:

Prompt templates: Prompt templates are templates for different types of prompts. Like “chatbot” style templates, ELI5 question-answering, etc
LLMs: Large language models like GPT-3, BLOOM, etc
Agents: Agents use LLMs to decide what actions should be taken. Tools like web search or calculators can be used, and all are packaged into a logical loop of operations.
Memory: Short-term memory, long-term memory.
We will dive into each of these in much more detail in upcoming chapters of the LangChain handbook.

For now, we’ll start with the basics behind prompt templates and LLMs. We’ll also explore two LLM options available from the library, using models from Hugging Face Hub or OpenAI.

In [1]:
!pip install langchain

In [12]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

# user question
question = "Which NFL team won the Super Bowl in the 2010 season?"

When using these prompt template with the given question we will get:

Question: Which NFL team won the Super Bowl in the 2010 season? Answer:

For now, that’s all we need. We’ll use the same prompt template across both Hugging Face Hub and OpenAI LLM generations.

# Hugging Face Hub LLM

In [11]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [14]:
!pip install huggingface_hub

In [3]:
from langchain import HuggingFaceHub, LLMChain


In [17]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':1e-10}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about NFL 2010
print(llm_chain.run(question))

san francisco 49ers


In [28]:
# ask the user question about NFL 2010
ans = llm_chain.run("Which NFL team won the Super Bowl in the 2010 season?")

In [29]:
ans

'san francisco 49ers'

# Asking Multiple Questions

# If we’d like to ask multiple questions, we can try two approaches:

If we’d like to ask multiple questions, we can try two approaches:

- Iterate through all questions using the generate method, answering them one at a time.

- Place all questions into a single prompt for the LLM; this will only work for more advanced LLMs.

 Starting with option (1), let’s see how to use the generate method:


In [30]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

LLMResult(generations=[[Generation(text='san francisco 49ers', generation_info=None)], [Generation(text='76', generation_info=None)], [Generation(text='john glenn', generation_info=None)], [Generation(text='one', generation_info=None)]], llm_output=None, run=[RunInfo(run_id=UUID('abf0ae55-0d60-4c2e-898b-d2e4af549ec1')), RunInfo(run_id=UUID('478cefba-d9f5-41e3-bc93-ab2b36c6e274')), RunInfo(run_id=UUID('601bab7a-2f6a-4daf-9fc7-9bd26b41a7b2')), RunInfo(run_id=UUID('4c3de90c-41ff-444b-a583-d04227904bce'))])

In [15]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)
print(llm_chain.run(qs_str))

 The Green Bay Packers won the Super Bowl in the 2010 season. I am 193 centimeters tall if I am 6 ft 4 inches. Edwin "Buzz" Aldrin was the 12th person on the moon. A blade of grass does not have eyes.


# OpenAI LLMs

The OpenAI endpoints in LangChain connect to OpenAI directly or via Azure. We need an OpenAI account and API key to use these endpoints.

Once you have an API key, we add it to the OPENAI_API_TOKEN environment variable. We can do this with Python like so:

In [4]:
import os

os.environ['OPENAI_API_KEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [5]:
!pip install openai

In [19]:
from langchain.llms import OpenAI

davinci = OpenAI(model_name = 'text-davinci-003')

In [20]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

 The Green Bay Packers won the Super Bowl in the 2010 season.


In [21]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
print(llm_chain.generate(qs))

generations=[[Generation(text=' The Green Bay Packers won Super Bowl XLV in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 193.04 cm', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' The twelfth person to walk on the moon was astronaut Harrison Schmitt on Apollo 17 on December 11, 1972.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have any eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 75, 'total_tokens': 126, 'completion_tokens': 51}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('e1ba5f35-e11d-4a70-b3fc-678293d2cfd3')), RunInfo(run_id=UUID('805b70af-6be1-4b23-bbf9-716d86c3dae6')), RunInfo(run_id=UUID('f31f722e-2747-4a4c-a4d7-7bdba139a1c5')), RunInfo(run_id=UUID('da7ad66b-dde5-45d9-b918-8bddb11c2f39'))]


In [16]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

In [22]:
llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The New Orleans Saints won the Super Bowl in the 2010 season.
You are 193 cm tall if you are 6 ft 4 inches.
The 12th person on the moon was Charles Duke.
A blade of grass has no eyes.
